# Imports and Pre-requisites

In [1]:
import ee
import geemap
from datetime import timedelta, datetime

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [2]:
ee.Authenticate()

True

In [6]:
ee.Initialize(project="rwanda-climate-alerts")

# Collect Datasets

## Baseline setup
In this section, we will define the
- baseline area of interest: polygon covering Rwanda
- drought/flood years

In [7]:
# drought_years = [
#     "1984-10-01",
#     "1989-12-01",
#     "1996-01-01",
#     "1999-11-01",
#     "2000-01-01",  # "Early 2000"
#     "2003-03-01",
#     "2005-02-01",
#     "2006-03-01",
#     "2006-09-01",
#     "2014-06-01"
# ]
#
# # Flood dates
# flood_years = [
#     "1988-05-06",
#     "2000-11-21",
#     "2001-09-22",
#     "2001-10-30",
#     "2002-04-26",
#     "2003-10-30",
#     "2005-08-16"
# ]

In [8]:
# Convert date string to date format
def str_to_date_range(date_string, date_format):
    initial_date = datetime.strptime(date_string, date_format)

    two_weeks_early_date = initial_date - timedelta(weeks=2)
    two_weeks_later_date = initial_date + timedelta(weeks=2)

    return two_weeks_early_date, two_weeks_later_date

# date_1, date_2 = str_to_date_range("1970-01-01", "%Y-%m-%d")
# print(date_1, date_2)

def date_range_to_list(date_list):
    date_range_list = []

    for flood_date in date_list:
        date_range_0, date_range_1 = str_to_date_range(flood_date, "%Y-%m-%d")
        date_range_list.append([date_range_0, date_range_1])

    return date_range_list

flood_dates_range = []

for flood_range in flood_dates_range:
    print(flood_range[0], flood_range[1])

In [9]:
# Drought dates
drought_years = ["1984", "1989", "1996", "1999", "2000", "2003", "2005", "2006", "2006", "2014"]

# Flood dates
flood_years = ["1988", "2000", "2001", "2001", "2002", "2003", "2005"]

## Fetch Datasets

In [10]:
# Create map
Map = geemap.Map()

# Fetch the district outline of Rwanda
districts = ee.FeatureCollection("FAO/GAUL/2015/level2") \
                .filter(ee.Filter.eq("ADM0_NAME", "Rwanda"))

districts_geometry = districts.map(lambda feature: feature.geometry())

# Polygon englobing Rwanda
rwanda = ee.Geometry.Polygon([
    [
        [28.70, -2.85],
        [31.00, -2.85],
        [31.00, -1.00],
        [28.70, -1.00]
    ]
])

# Add a buffer around Rwanda
rwanda_buffered = rwanda.buffer(10000) # 10,000 meters
Map.centerObject(districts, 7)

In [11]:
# CHIRPS Daily Rainfall (mm/day)
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
            .filterDate("2005-01-01", "2025-07-31")\
            .filterBounds(rwanda_buffered) \
            .map(lambda img: img.clip(rwanda_buffered))

# ERA5 Land Monthly Temperature
era5_temp = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
                .select("temperature_2m") \
                .filterDate("2005-01-01", "2025-07-31") \
                .filterBounds(rwanda_buffered) \
                .map(lambda img: img.clip(rwanda_buffered))

# Soil Moisture (ERA5 Land)
soil_moist = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
                .select("volumetric_soil_water_layer_1") \
                .filterDate("2005-01-01", "2025-07-31") \
                .filterBounds(rwanda_buffered) \
                .map(lambda img: img.clip(rwanda_buffered))

# MODIS NDVI (monthly, 250 m)
ndvi = ee.ImageCollection("MODIS/006/MOD13A1") \
            .select("NDVI") \
            .filterDate("2005-01-01", "2025-07-31") \
            .filterBounds(rwanda_buffered) \
            .map(lambda img: img.clip(rwanda_buffered))

# DEM (SRTM ~30m resolution)
dem = ee.Image("USGS/SRTMGL1_003").clip(rwanda_buffered)

# Calculate slope (degrees)
slope = ee.Terrain.slope(dem)


# ESA WorldCover 2020 (10 m)
# landcover = ee.Image("ESA/WorldCover/v200/2022").clip(rwanda_buffered)

# MODIS Land Cover (optional, for time series)
# modis_lc = ee.ImageCollection("MODIS/006/MCD12Q1") \
#                 .select("LC_Type1") \
#                 .filterDate("2001-01-01", "2020-12-31") \
#                 .filterBounds(districts) \
#                 .map(lambda img: img.clip(rwanda_buffered))

In [12]:
# print(chirps.bandNames().getInfo())
# print(era5_temp.bandNames().getInfo())
# print(soil_moist.bandNames().getInfo())
# print(ndvi.bandNames().getInfo())
# print(dem.bandNames().getInfo())

# Aggregate monthly

In [13]:
def aggregate_monthly(image_collection):
    monthly_result = image_collection.map(lambda img: img.set("month", img.date().format("YYYY-MM")))
    monthly_sum = monthly_result.reduce(ee.Reducer.sum())
    return monthly_result, monthly_sum

monthly_chirps, monthly_chirps_sum = aggregate_monthly(chirps)
monthly_era5_temp, monthly_era5_temp_sum = aggregate_monthly(era5_temp)
monthly_soil_moist, monthly_soil_moist_sum = aggregate_monthly(soil_moist)
monthly_ndvi, monthly_ndvi_sum = aggregate_monthly(ndvi)

# print(type(monthly_era5_temp_sum), type(monthly_chirps_sum))
# print(monthly_chirps.getInfo())

# Calculate historical baseline and anomalies

In [14]:
def calculate_baseline(image_collection):
    # Define baseline period (e.g., 2000–2015)
    baseline = image_collection.filterDate("2005-01-01", "2015-12-31")

    # Mean rainfall baseline
    baseline_mean = baseline.mean()

    # Recent period (e.g., 2020–2025)
    recent = image_collection.filterDate("2025-07-01", "2025-07-31").mean()

    # Rainfall anomaly (recent vs baseline)
    anomaly = recent.subtract(baseline_mean).divide(baseline_mean)

    return baseline, baseline_mean, anomaly

rain_baseline, rain_baseline_mean, rain_anomaly = calculate_baseline(chirps)
temp_baseline, temp_baseline_mean, temp_anomaly = calculate_baseline(era5_temp)
soil_moist_baseline, soil_moist_baseline_mean, soil_moist_anomaly = calculate_baseline(soil_moist)
ndvi_baseline, ndvi_baseline_mean, ndvi_anomaly = calculate_baseline(ndvi)

# print(type(ndvi_anomaly))
# print(ndvi_anomaly.getInfo())

# Normalize indices (0-1)

In [15]:
def normalize(img, region):
    min_dict = img.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=region,
        scale=1000,
        maxPixels=1e13,
        bestEffort=True
    ).values().get(0)
    max_dict = img.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=region,
        scale=1000,
        maxPixels=1e13,
        bestEffort=True
    ).values().get(0)

    min_value = ee.Number(min_dict)
    max_value = ee.Number(max_dict)

    return img.subtract(min_value).divide(max_value.subtract(min_value))
    # range_value = max_value.subtract(min_value)
    #
    # # Protect against divide by zero (flat images)
    # norm_img = ee.Algorithms.If(
    #     range_value.eq(0),
    #     img.multiply(0),  # return constant zero image
    #     img.subtract(min_value).divide(range_value)
    # )
    # return ee.Image(norm_img).rename(img.bandNames().get(0))

rain_norm = normalize(rain_anomaly, districts)
temp_norm = normalize(temp_anomaly, districts)
soil_moist_norm = normalize(soil_moist_anomaly, districts)
ndvi_norm = normalize(ndvi_anomaly, districts)

dem_norm = normalize(dem, districts)
slope_norm = normalize(slope, districts)

# print(type(slope))
# print(dem_norm.getInfo())

# Cobine risk indexes

In [16]:
flood_risk_index = rain_norm.multiply(0.4) \
                    .add(temp_norm.multiply(0.3)) \
                    .add(slope_norm.multiply(0.2))
                    # .add(ndvi_norm.multiply(0.2)) \

drought_risk_index = rain_norm.multiply(-1).add(1).multiply(0.4) \
                    .add(soil_moist_norm.multiply(-1).add(1).multiply(0.3)) \
                    .add(temp_norm.multiply(0.3))
                    # .add(ndvi_norm.multiply(-1).add(1).multiply(0.2)) \
# NDVI is causing some trouble, still in the process of figuring it out

landslide_risk_index = rain_norm.multiply(0.4) \
                        .add(soil_moist_norm.multiply(0.3)) \
                        .add(slope_norm.multiply(0.3))

# print(landslide_risk_index.getInfo())

# Aggregate risk by district

In [17]:
def aggregate_risk(risk_index):
    district_stats = risk_index.reduceRegions(
        collection=districts,
        reducer=ee.Reducer.mean(),
        scale=1000
    )
    return district_stats

flood_risk_stats = aggregate_risk(flood_risk_index)
drought_risk_stats = aggregate_risk(drought_risk_index)
landslide_risk_stats = aggregate_risk(landslide_risk_index)

# print(type(flood_risk_stats))
# Print first feature
# print(flood_risk_stats.first().getInfo())

In [23]:
vis_params = {"min": 0, "max": 1, "palette": ["green", "yellow", "red"]}
Map.addLayer(flood_risk_index, vis_params, "Flood Risk Index")
Map.addLayer(drought_risk_index, vis_params, "Drought Risk Index")
Map.addLayer(landslide_risk_index, vis_params, "Landslide Risk Index")

Map.addLayer(bugesera, {"color": "black"}, "Bugesera")

# Add district boundaries
Map.addLayer(districts, {"color": "black"}, "Districts")

Map

Map(bottom=33393.0, center=[-1.7849895078809694, 29.932265099545248], controls=(WidgetControl(options=['positi…